In [35]:
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpStatus
import pandas as pd

In [29]:
df_vehiculos = pd.read_excel('../../Datos_P1/df_vehicle.xlsx')
df_clientes = pd.read_excel('../../Datos_P1/df_orders.xlsx')
df_distancias = pd.read_excel('../../Datos_P1/df_distance_km.xlsx')

df_distancias.index = df_distancias.columns

,Cliente_1,Cliente_2,Cliente_3,Cliente_4,Cliente_5,Cliente_6,Cliente_7,Cliente_8,Cliente_9,Cliente_10,...,Cliente_12,Cliente_13,Cliente_14,Cliente_15,Cliente_16,Cliente_17,Cliente_18,Cliente_19,Cliente_20,Almacén
Cliente_1,0.0000,7.5625,15.5365,1.1998,4.7145,1.7407,7.9408,17.1947,4.2933,3.2659,...,6.0225,5.4470,2.2133,11.1505,1.5775,10.8288,9.1456,20.4871,22.1445,3.6114
Cliente_2,7.5625,0.0000,3.3838,7.7433,14.5720,8.5237,0.4847,13.7974,10.1522,7.1521,...,10.1049,2.6961,13.4907,18.0835,7.0275,19.8218,8.2737,9.6369,19.1038,10.7361
Cliente_3,15.5365,3.3838,0.0000,12.5438,0.0000,0.0000,0.0000,16.0355,13.9120,13.0649,...,12.3430,5.0114,15.7289,17.9217,9.6824,22.0599,10.5118,7.7574,16.5997,13.9021
Cliente_4,1.1998,7.7433,12.5438,0.0000,5.0721,0.9119,7.5798,17.4095,3.5781,3.3451,...,6.2330,4.7117,2.8799,11.3610,1.3127,11.1926,9.3561,20.7019,21.1518,3.3673
Cliente_5,4.7145,14.5720,0.0000,5.0721,0.0000,4.8187,0.0000,0.0000,0.0000,7.2170,...,6.8738,9.1064,3.6476,12.0019,5.4347,5.2872,9.9969,21.3384,25.5947,4.5417
Cliente_6,1.7407,8.5237,0.0000,0.9119,4.8187,0.0000,7.8866,20.0472,2.9060,4.0899,...,8.8750,5.0185,3.2185,14.0030,2.0575,11.4008,11.9981,19.7467,20.4797,2.6952
Cliente_7,7.9408,0.4847,0.0000,7.5798,0.0000,7.8866,0.0000,13.9893,10.1499,7.3440,...,10.2968,0.0000,0.0000,0.0000,7.2194,20.0138,8.4656,9.8289,17.1722,10.9280
Cliente_8,17.1947,13.7974,16.0355,17.4095,0.0000,20.0472,13.9893,0.0000,19.6963,12.9161,...,12.3127,13.5906,14.8623,5.9742,14.6038,22.6587,8.9984,15.5515,34.1368,20.8703
Cliente_9,4.2933,10.1522,13.9120,3.5781,0.0000,2.9060,10.1499,19.6963,0.0000,6.8679,...,10.1024,7.7229,4.5536,15.2304,4.8355,9.7565,13.2255,24.5670,22.9781,1.0494
Cliente_10,3.2659,7.1521,13.0649,3.3451,7.2170,4.0899,7.3440,12.9161,6.8679,0.0000,...,2.3061,4.9896,0.0000,8.2196,0.0000,10.8948,5.3050,17.5604,21.1407,5.9516


In [27]:
# Preparación datos para PuLP

capacidades = dict(zip(df_vehiculos["vehiculo_id"], df_vehiculos["capacidad_kg"]))
costos = dict(zip(df_vehiculos["vehiculo_id"], df_vehiculos["costo_km"]))
autonomias = dict(zip(df_vehiculos["vehiculo_id"], df_vehiculos["autonomia_km"]))

pedidos = dict(zip(df_clientes["cliente"], df_clientes["order_demand"]))
#pedidos["Almacén"] = 0

distancias = df_vehiculos.stack().to_dict()  # Convierte a un diccionario (i, j): distancia

clientes = df_clientes["cliente"].tolist() #+ ["Almacén"]
vehiculos = df_vehiculos["vehiculo_id"].tolist()

In [ ]:
# Crear el problema
problema = LpProblem("Ruteo_de_Vehiculos", LpMinimize)


# Variables de decisión
x = LpVariable.dicts("x", [(i, j, k) for i in clientes for j in clientes for k in vehiculos], cat="Binary")
q = LpVariable.dicts("q", [(i, k) for i in clientes for k in vehiculos], lowBound=0)

# Función objetivo: minimizar costo total
problema += lpSum(
    x[i, j, k] * costos[k] * distancias.get((i, j), 0)
    for i in clientes for j in clientes for k in vehiculos
)

# Restricciones
# Cada cliente debe ser atendido una vez
for i in clientes:
    problema += lpSum(x[i, j, k] for j in clientes for k in vehiculos) == 1

# Respetar la capacidad de los vehículos
for k in vehiculos:
    problema += lpSum(q[i, k] for i in clientes) <= capacidades[k]

# Satisfacer la demanda de los clientes
for i in clientes:
    problema += lpSum(q[i, k] for k in vehiculos) == pedidos[i]

# Autonomía de los vehículos
for k in vehiculos:
    problema += lpSum(x[i, j, k] * distancias.get((i, j), 0) for i in clientes for j in clientes) <= autonomias[k]

# Flujo de vehículos: entrada = salida
for i in clientes:
    for k in vehiculos:
        problema += lpSum(x[i, j, k] for j in clientes) == lpSum(x[j, i, k] for j in clientes)

# Resolver el problema
problema.solve()
print(LpStatus[problema.status])
# Resultados
for k in vehiculos:
    print(f"Vehículo {k}:")
    for i in clientes:
        for j in clientes:
            if x[i, j, k].value() == 1:
                print(f"  {i}")

Vehículo 1:
  Cliente_1
  Cliente_4
  Cliente_15
  Cliente_16
Vehículo 2:
  Cliente_7
Vehículo 3:
  Cliente_2
  Cliente_12
  Cliente_18
Vehículo 4:
  Cliente_3
  Cliente_5
Vehículo 5:
  Cliente_6
  Cliente_8
  Cliente_9
  Cliente_10
  Cliente_11
  Cliente_17
  Cliente_19
Vehículo 6:
  Cliente_13
  Cliente_14
  Cliente_20


In [34]:
df_clientes

,cliente,mes_anio,order_demand
0,Cliente_1,12-2024,909
1,Cliente_2,12-2024,959
2,Cliente_3,12-2024,960
3,Cliente_4,12-2024,980
4,Cliente_5,12-2024,979
5,Cliente_6,12-2024,908
6,Cliente_7,12-2024,924
7,Cliente_8,12-2024,920
8,Cliente_9,12-2024,886
9,Cliente_10,12-2024,964
